# Merge catalogs
Reconcile full catalog on hal/newton in Fall 2021 with the incomplete but partially reclassified catalog from my laptop whilst in Paris in July 2021
Need to reconcile reawav_MVOE_all.csv (concatenated reawav_MVOE_YYYYMM.csv) on hal9000 against catalog_unique.csv which came from processing in Paris 
on my laptop.

Need to merge the new_subclass, weight, checked, split, delete and ignore columns.
Should probably match on the WAVfile path.
Do I need to merge D, R, r, e, l, h and t columns?

In [29]:
import os
import sys
import pandas as pd
SEISAN_DATA = os.path.join( os.getenv('HOME'),'DATA','MVO')
SEISAN_DB = 'MVOE_'

def fix_df(df):
    correct_columns = ['filetime', 'Fs', 'bandratio_[0.8_4.0_16.0]',
       'bandratio_[1.0_6.0_11.0]', 'bw_max', 'bw_min', 'calib',
       'cft_peak_wmean', 'cft_std_wmean', 'coincidence_sum', 'day',
       'detection_quality', 'energy', 'hour', 'kurtosis', 'medianF', 'minute',
       'month', 'noise_level', 'num_gaps', 'num_traces', 'offtime', 'ontime',
       'path', 'peakA', 'peakF', 'peakamp', 'peaktime', 'percent_availability',
       'quality', 'sample_lower_quartile', 'sample_max', 'sample_mean',
       'sample_median', 'sample_min', 'sample_rms', 'sample_stdev',
       'sample_upper_quartile', 'second', 'sfile', 'signal_level', 'skewness',
       'snr', 'subclass', 'trigger_duration', 'year', 'D', 'R',
       'r', 'e', 'l', 'h', 't', 'new_subclass', 'weight', 'checked', 'split',
       'delete', 'ignore'] # removed starttime
    subset_columns = []
    for column in df.columns:
        if column in correct_columns:
            subset_columns.append(column)
    print(len(subset_columns))
    #df = df[subset_columns] # subset to correct columns
    #df.set_index('path', inplace=True) # try this
    #df.sort_index(inplace=True)   
    df2 = df[subset_columns] # subset to correct columns
    df3 = df2.set_index('path', inplace=False) # try this
    df4 = df3.sort_index(inplace=False)    
    return df4

def df2csv_without_index(df, csvfile):
    df = df.reset_index()  
    df.drop(df.filter(regex="Unname"),axis=1, inplace=True)
    df.to_csv(csvfile, index=False)

def print_checked_rows(): # not used
    rows = []
    for i,row in unchecked2.iterrows():
        if row['checked']:
            rows.append(row)
    print(pd.DataFrame(rows))    
    
full_unchecked_csv = os.path.join(SEISAN_DATA, 'MachineLearning', SEISAN_DB, 'labelling', '%scatalog.csv' % SEISAN_DB)
full_unchecked_cat = pd.read_csv(full_unchecked_csv)    
unchecked = fix_df(full_unchecked_cat)
unchecked.head()
unchecked2 = unchecked.copy()

old_checked_csv = "../CSVfiles/catalog_unique.csv"
old_checked_cat = pd.read_csv(old_checked_csv)
checked = fix_df(old_checked_cat)
checked.head()
checked2 = checked[['quality','subclass','D', 'R',
       'r', 'e', 'l', 'h', 't', 'new_subclass', 'weight', 'checked', 'split',
       'delete', 'ignore']]

# MERGE
unchecked2.update(checked2)

# WRITE
if len(unchecked)==len(unchecked2):    
    df2csv_without_index(unchecked2, 'merged_catalog.csv')
    # manually copy this to ~/DATA/MVO/MachineLearning/SEISAN_DB/original/merged_catalog.csv